In [1]:
%load_ext autoreload
%autoreload 2
import dspy
import os
os.environ['OPENAI_API_KEY'] = 'sk-...'

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=4000)
gpt4 = dspy.OpenAI(model='gpt-4', max_tokens=4000)
dspy.settings.configure(lm=turbo)

In [3]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 50)

In [4]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [5]:
from dspy.evaluate import Evaluate
from dspy.evaluate.metrics import answer_exact_match
from dspy.functional import TypedPredictor
from dspy.teleprompt.signature_opt2 import optimize_signature

evaluator = Evaluate(devset=devset, metric=answer_exact_match, num_threads=10, display_progress=True)

program = optimize_signature(
    student=TypedPredictor(BasicQA),
    evaluator=Evaluate(devset=trainset, metric=answer_exact_match, num_threads=10, display_progress=True),
    initial_prompts=2,
    n_iterations=8,
    verbose=True,
    prompt_model=gpt4,
)

None


/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:184: UserWarning: Field name "signature" shadows an attribute in parent "Signature"; 
  warnings.warn(


TypeError: Field 'signature' in 'GenerateInstructionGivenAttempts' must be declared with InputField or OutputField. field.json_schema_extra=None

In [ ]:
gpt4.inspect_history(n=3)





Given the fields `attempted_signatures`, produce the fields `proposed_signature`.

---

Follow the following format.

Attempted Signatures: ${attempted_signatures}
Reasoning: Let's think step by step in order to ${produce the proposed_signature}. We ...
Proposed Signature: The improved signature for the language model. Respond with a single JSON object. JSON Schema: {"properties": {"instructions": {"description": "The instructions for the task", "title": "Instructions", "type": "string"}, "question_prefix": {"description": "The prefix for question", "title": "Question Prefix", "type": "string"}, "question_desc": {"description": "The description for question", "title": "Question Desc", "type": "string"}, "answer_prefix": {"description": "The prefix for answer", "title": "Answer Prefix", "type": "string"}, "answer_desc": {"description": "The description for answer", "title": "Answer Desc", "type": "string"}}, "required": ["instructions", "question_prefix", "question_desc", "answer_pr